In [ ]:
%matplotlib inline

# Co-simulation with a serial master algorithm.

Sometimes,
we may want to simulate a system of several FMU models coupled together.
[TimeSteppingSystem][gemseo_fmu.disciplines.time_stepping_system.TimeSteppingSystem]
allows to perform this _co-simulation_ task,
with a parallel master algorithm based on the Jacobi method.
In this example,
we will see how to replace this master algorithm by a serial one
using the Gauss-Seidel method.


In [ ]:
from __future__ import annotations

from matplotlib import pyplot as plt

from gemseo_fmu.disciplines.fmu_discipline import FMUDiscipline
from gemseo_fmu.disciplines.time_stepping_system import TimeSteppingSystem
from gemseo_fmu.problems.fmu_files import get_fmu_file_path

Let us consider a set of two mass-spring pairs connected to each other
and modelled by two FMU models:

$$
\begin{cases}
x_1' = v_1\\
v_1' = -\frac{k_1+k_2}{m_1}x_1+\frac{k_2}{m_1}x_2
\end{cases}
$$

and

$$
\begin{cases}
x_2' = v_2\\
v_2' = -\frac{k_2+k_3}{m_2}x_2+\frac{k_2}{m_2}x_1
\end{cases}
$$

These models can be co-simulated by instantiating a
[TimeSteppingSystem][gemseo_fmu.disciplines.time_stepping_system.TimeSteppingSystem]:



In [ ]:
system = TimeSteppingSystem(
    (
        get_fmu_file_path("MassSpringSubSystem1"),
        get_fmu_file_path("MassSpringSubSystem2"),
    ),
    50,
    0.01,
    mda_name="MDAGaussSeidel",
)

Note that in this case,
we do not use the default MDA name `"MDAJacobi"` implementing a Jacobi method
but `"MDAJacobi"` implementing a Gauss-Seidel technique.
The disciplines are no longer executed in parallel but sequentially.

Then wew can execute this system from initial time to final time:



In [ ]:
system.execute()

or with time stepping by setting `do_step` to `False` at instantiation.
For this particular example,
we also have a FMU model of the complete system:



In [ ]:
reference = FMUDiscipline(
    get_fmu_file_path("MassSpringSystem"), final_time=50, time_step=0.01
)
reference.execute()

Then,
we can compare the solutions graphically
in terms of position and velocity of the two masses
and note that for this example,
the co-simulation of the two subsystems is equivalent
to that of the complete system.



In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1)
time_1 = system.local_data["MassSpringSubSystem1:time"]
time_2 = system.local_data["MassSpringSubSystem2:time"]
ax1.plot(time_1, system.local_data["x1"], label="x1", color="red")
ax1.plot(time_2, system.local_data["x2"], label="x2", color="blue")
ax2.plot(time_1, system.local_data["v1"], label="v1", color="red")
ax2.plot(time_2, system.local_data["v2"], label="v2", color="blue")

time = reference.local_data["MassSpringSystem:time"]
ax1.plot(time, reference.local_data["x1"], label="x1[ref]", linestyle="--", color="red")
ax1.plot(
    time, reference.local_data["x2"], label="x2[ref]", linestyle="--", color="blue"
)
ax2.plot(time, reference.local_data["v1"], label="v1[ref]", linestyle="--", color="red")
ax2.plot(
    time, reference.local_data["v2"], label="v2[ref]", linestyle="--", color="blue"
)

ax1.set_xlabel("Time (s)")
ax1.set_ylabel("Position (m)")
ax1.grid()
ax1.legend()
ax2.set_xlabel("Time (s)")
ax2.set_ylabel("Velocity (m/s)")
ax2.grid()
ax2.legend()
plt.show()